In [ ]:
import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import numpy as np
import seaborn as sns

import network_maps

import preprocess_mikrotik_starlink

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
plt.rcParams["figure.dpi"] = 200
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
fig_size = (32, 8)

x=3
y=3
fig_size = (x*y, y)

# Make Matplotlib's automatic date tick formatting show time
mpl.rcParams['date.autoformatter.hour'] = '%H:%M:%S'
mpl.rcParams['date.autoformatter.minute'] = '%H:%M:%S'
mpl.rcParams['date.autoformatter.second'] = '%H:%M:%S'
mpl.rcParams['date.autoformatter.microsecond'] = '%H:%M:%S.%f'  # optional

In [ ]:
data_dir = 'data/2025-05-06-wan/'

list_of_files = os.listdir(data_dir)
list_of_files.sort()
num_files = len(list_of_files)
print('Number of files: {}'.format(num_files))

df_list = []
for file in list_of_files:
    # Skipping the files we're not using
    if file[-5:] != ".gzip": # .gzip
        continue
    temp_df = pd.read_parquet(data_dir+file)
    df_list.append(temp_df)
df = pd.concat(df_list)

# Infer objects, then convert dtypes
df = df.infer_objects().convert_dtypes()

for column in df.columns:
        try:
            df[column] = pd.to_numeric(df[column])
        except (ValueError, TypeError):
            pass  # Skip columns that cannot be convertedmq

# Rename timestamp from Python and keep it for future use
# It is unreliable if a lot of messages come at the same time due to congestion
df['timestamp_python'] = df.pop('@timestamp')

# It is better to rely on timestamps from the router rather than ElasticSearch
df['timestamp_router'] = pd.to_datetime(df['date'] + ' ' +  df['time'])
df.drop(columns=['date', 'time'], inplace=True)
df = df.sort_values(by=['timestamp_router'])
df = df.set_index('timestamp_router', drop=False)
#df = df.reset_index()

# Convert Data Class into integer mapping
dataclass_mapping = {'': 0, 'LTE': 1, '5G NSA': 2, '5G SA': 3}
df['lte.lDataClassInt'] = df['lte.lDataClass'].map(dataclass_mapping)

# Convert modulation into fixed bits per hz mapping
modulation_mapping = {'': 0, 'qpsk': 2, '16qam': 4, '64qam': 6, '256qam': 8}
df['lte.lDlModulationInt'] = df['lte.lDlModulation'].map(modulation_mapping)
df['lte.lNrDlModulationInt'] = df['lte.lNrDlModulation'].map(modulation_mapping)

# Fix wrong scaling on Rsrq and NrRsrq
# If the value is -12dB, it is shown as -120
df['lte.lRsrq'] = df['lte.lRsrq'] / 10
df['lte.lNrRsrq'] = df['lte.lNrRsrq'] / 10

# Create handover events
df['handover_CurrentCellid'] = df['lte.lCurrentCellid'].diff().ne(0).astype(float)

print('df before')
df.info(verbose=True, show_counts=True, memory_usage='deep')

# Compress float64/Float64 to float32 and int64/Int64 to int32
dtype_mapping = {col: 'float32' for col in df.select_dtypes(include=['float64', 'Float64']).columns}
dtype_mapping.update({col: 'int32' for col in df.select_dtypes(include=['int64', 'Int64']).columns})

df = df.astype(dtype_mapping)

print('df after')
df.info(verbose=True, show_counts=True, memory_usage='deep')

#df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.keys()

In [ ]:
df['identity'].unique()

In [ ]:
df.columns = df.columns.str.removeprefix("gps.")
df.columns = df.columns.str.removeprefix("lte.")
df.info()

In [ ]:
df['lDataClass'].unique()

In [ ]:
df['lDataClass'].value_counts()

In [ ]:
df['lPrimaryBand'].unique()

In [ ]:
#df = df.between_time('11:41', '12:00')
# start_date = '2025-01-22 09:00:00'
# end_date = '2025-01-22 11:30:00'
# mask = (df['timestamp_router'] > start_date) & (df['timestamp_router'] <= end_date)
# df = df.loc[mask] 

In [ ]:
def get_PrimaryBandMHzNumber_v2(df):
    # Extract 'PrimaryBand' and 'PrimaryBandMHz' directly using regex
    df[['PrimaryBandName', 'PrimaryBandMHz']] = df['lPrimaryBand'].str.extract(r'(\S+)@(\d+Mhz)')
    
    # Drop rows where 'PrimaryBandMHz' is NaN
    #df.dropna(subset=['PrimaryBandMHz'], inplace=True)
    
    # Convert 'PrimaryBandMHz' to integer after removing 'Mhz'
    df['PrimaryBandMHz'] = df['PrimaryBandMHz'].str.replace('Mhz', '').astype(int)
    
    # Drop the intermediate columns
    df.drop(columns=['lPrimaryBand'], inplace=True)
    
    return df

In [ ]:
def get_PrimaryBandMHzNumber_v3(df):
    # Extract 'PrimaryBand' and 'PrimaryBandMHz' using regex
    df[['PrimaryBandName', 'PrimaryBandMHz']] = df['lPrimaryBand'].str.extract(r'(\S+)@(\d+)Mhz')
    
    # Convert 'PrimaryBandMHz' to integer, handling NaN values
    df['PrimaryBandMHz'] = df['PrimaryBandMHz'].astype('Int64')  # Nullable integer type
    
    # Drop the original column
    df.drop(columns=['lPrimaryBand'], inplace=True)
    
    return df


In [ ]:
df.info()

In [ ]:
df = get_PrimaryBandMHzNumber_v3(df)

In [ ]:
df['speed'] = pd.to_numeric(df['speed'].str.split().str[0])

In [ ]:
df['speed'].describe()

In [ ]:
df['lDataClass'].unique()

In [ ]:
df.head(10)

In [ ]:
df['identity'].unique()

In [ ]:
df_D2 = df.query("identity == '5G-D2-WAVELAB'")
df_DTAG = df.query("identity == '5G-DTAG-WAVELAB'")
df_4329 = df.query("identity == 'CAU-4329'")
df_0C = df.query("identity == 'CAU-0C'")

In [ ]:
df_D2['lDataClass'].unique()

In [ ]:
ax = df_D2[['ltxbitspersecond']].div(1024*1024).plot(figsize=(40,16))#, ylim=(0,30))
df_D2[['handover_CurrentCellid']].plot(ax=ax)
df_D2[['lDataClassInt']].plot(ax=ax)
df_D2[['PrimaryBandMHz']].plot(ax=ax)
#df_D2[['lCqi']].plot(ax=ax)
df_D2[['lSinr']].plot(ax=ax)

# ax = df_vodafone[['tx_rate_mbits']].plot(figsize=(40,2), ylim=(0,7))
# ax = df_D2[['handover_CurrentCellid']].plot(figsize=(40,2))
# ay = df_D2[['lDataClassInt']].plot(figsize=(40,2))
# ax = df_D2[['lCqi']].plot(figsize=(40,2))
# ay = df_D2[['lSinr']].plot(figsize=(40,2))
# #style="o"

In [ ]:
df_D2['PrimaryBandMHz'].plot(figsize=fig_size)

In [ ]:
df_DTAG['lDataClass'].unique()

In [ ]:
ax = df_D2[['lDataClassInt']].plot(figsize=fig_size, style='.-', title='D2')
ay = df_DTAG[['lDataClassInt']].plot(figsize=fig_size, style='.-', title='DTAG')

In [ ]:
ax = df_D2[['lCqi']].plot(figsize=fig_size, style='.-', title='D2')
ay = df_DTAG[['lCqi']].plot(figsize=fig_size, style='.-', title='DTAG')

In [ ]:
# SINR Range: -20 to 30 dB in LTE, -23 to 40 dB in 5G NR.

# <SINR>
# In LTE mode:
# It indicates LTE Signal-to-Interface plus Noise Ratio. 
# The conversion formula for actual SINR is 
# Y = (1/5) x X x 10 - 20 
# (X is the <SINR> value queried by AT+QENG and 
# Y is the actual value of LTE SINR after calculating with the formula). 
# Range: -20 to 30 dB.
# In 5G NR mode:
# It indicates the signal of 5G NR Signal-to-Interface plus Noise Ratio.
# Range: -20 to 30 dB.
df_D2[['lSinr']].describe()

In [ ]:
# Define the valid SINR range
sinr_min = -20  # Minimum SINR value
sinr_max = 30   # Maximum SINR value

df_D2['SINR_clipped'] = df_D2['lSinr'].clip(lower=-20, upper=30)

# Normalize SINR to range [0, 1]
df_D2['SINR_normalized'] = (df_D2['SINR_clipped'] - sinr_min) / (sinr_max - sinr_min)

In [ ]:
aa = df_D2[['lSinr']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
aa = df_D2[['SINR_normalized']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
# <RSRP> 16-bit signed integer.
# In LTE mode:
# It indicates the signal of LTE Reference Signal Received Power (see 3GPP 36.214). 
# Range: -140 to -44 dBm. The closer to -44, the better the signal is.
# The closer to -140, the worse the signal is.
# In 5G NR mode:
# It indicates the signal of 5G NR Reference Signal Received Power. 
# Range: -140 to -44 dBm. The closer to -44, the better the signal is. 
# The closer to -140, the worse the signal is.
df_D2[['lRsrp']].describe()

In [ ]:
# Define the valid RSRP range
rsrp_min = -140  # Minimum RSRP value
rsrp_max = -44   # Maximum RSRP value

df_D2['RSRP_clipped'] = df_D2['lRsrp'].clip(lower=rsrp_min, upper=rsrp_max)

# Normalize RSRP to range [0, 1]
df_D2['RSRP_normalized'] = (df_D2['RSRP_clipped'] - rsrp_min) / (rsrp_max - rsrp_min)

In [ ]:
aa = df_D2[['lRsrp']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
aa = df_D2[['RSRP_normalized']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
# <RSRQ> 
# In LTE mode:
# It indicates the signal of current LTE Reference Signal Received Quality (see 3GPP 36.214). 
# Range: -20 to -3 dB. The closer to -3, the better the signal is. 
# The closer to -20, the worse the signal is.
# In 5G NR mode:
# It indicates the signal of current 5G NR Reference Signal Received Quality.
# Range: -20 to -3 dB. 
# The closer to -3, the better the signal is. 
# The closer to -20, the worse the signal is. 
df_D2[['lRsrq']].describe()

In [ ]:
# Define the valid RSRQ range
rsrq_min = -20  # Minimum RSRQ value
rsrq_max = -3   # Maximum RSRQ value

df_D2['RSRQ_clipped'] = df_D2['lRsrq'].clip(lower=rsrq_min, upper=rsrq_max)

# Normalize RSRQ to range [0, 1]
df_D2['RSRQ_normalized'] = (df_D2['RSRQ_clipped'] - rsrq_min) / (rsrq_max - rsrq_min)

In [ ]:
aa = df_D2[['lRsrq']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
aa = df_D2[['RSRQ_normalized']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
df_D2[['ltxbitspersecond']].div(1024*1024).describe()

In [ ]:
df_DTAG[['ltxbitspersecond']].div(1024*1024).describe()

In [ ]:
fig, ax = plt.subplots(figsize=(40, 10))

df_D2['speed'].rename('D2-GPS-speed').plot(ax=ax, )
df_DTAG['speed'].rename('DTAG-GPS-speed').plot(ax=ax, )

ax.set_title('GPS Speed over time')
ax.set_ylabel('GPS Speed (km/h)')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(40, 10))

df_D2['ltxbitspersecond'].div(1024*1024).rename('D2-Tx').plot(ax=ax, )
df_DTAG['ltxbitspersecond'].div(1024*1024).rename('DTAG-Tx').plot(ax=ax, )

ax.set_title('Tx Rate over time')
ax.set_ylabel('Tx Rate [Mbps]')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
# repetition of above for clarity
aa = df_D2[['ltxbitspersecond']].div(1024*1024).plot(figsize=fig_size, style='.-', label='D2')
ab = df_D2[['SINR_normalized']].plot(figsize=fig_size, style='.-', label='D2')
ac = df_D2[['RSRP_normalized']].plot(figsize=fig_size, style='.-', label='D2')
ad = df_D2[['RSRQ_normalized']].plot(figsize=fig_size, style='.-', label='D2')

In [ ]:
df_D2_selected = df_D2[['lDataClass',
            'lCqi',

            'lRsrp',
            'RSRP_normalized',
            
            'lRsrq',
            'RSRQ_normalized',

            'lSinr',
            'SINR_normalized',

            'ltxbitspersecond',
            'ltxpacketspersecond',
            ]].select_dtypes(include='number')


df_D2_selected.info()

In [ ]:
df_D2.drop(df_D2[df_D2['lat'] == 'none'].index, inplace = True)
df_D2.drop(df_D2[df_D2['lon'] == 'none'].index, inplace = True)

In [ ]:
df_D2['lat'] = pd.to_numeric(df_D2['lat'])
df_D2['lon'] = pd.to_numeric(df_D2['lon'])

In [ ]:
df_D2['PrimaryBandName'].value_counts()

In [ ]:
df_D2['PrimaryBandMHz'].value_counts()

In [ ]:
df_D2['lDataClass'].value_counts()

In [ ]:
df_D2['lDataClass'].value_counts(normalize=True) * 100

In [ ]:
df_D2_LTE = df_D2.query("`lDataClass` == 'LTE'") # no LTE data
df_D2_5G_NSA = df_D2.query("`lDataClass` == '5G NSA'") # no 5G NSA data
df_D2_5G_SA = df_D2.query("`lDataClass` == '5G SA'")

In [ ]:
#df_D2_LTE.info()

In [ ]:
#df_D2_5G_NSA.info()

In [ ]:
df_D2_5G_SA.info()

In [ ]:
#network_maps.create_coverage_squares_with_metric(df_D2, 'D2', 'auto', '2025-05-06', 'lCqi', lon_step = 0.0002, lat_step = 0.0001)
#network_maps.create_coverage_squares_with_metric(df_D2_LTE, 'D2', 'LTE', '2025-05-06', 'lCqi', lon_step = 0.0002, lat_step = 0.0001)
#network_maps.create_coverage_squares_with_metric(df_D2_5G_NSA, 'D2', '5G-NSA', '2025-05-06', 'lCqi', lon_step = 0.0002, lat_step = 0.0001)
network_maps.create_coverage_squares_with_metric(df_D2_5G_SA, 'D2', '5G-SA', '2025-05-06', 'lCqi', lon_step = 0.0002, lat_step = 0.0001)

In [ ]:
# List of directories to iterate through
data_dirs = [
    'data/2025-05-06-starlink/',
]

df_list = []

starlink_selected_columns = [
    '@timestamp',
    'dish_status.state',
    'dish_status.pop_ping_drop_rate',
    
    'dish_status.downlink_throughput_bps',
    'dish_status.uplink_throughput_bps',
    'dish_status.pop_ping_latency_ms',

    'dish_status.fraction_obstructed',
    'dish_status.currently_obstructed',
    'dish_status.is_snr_above_noise_floor',
    
    'dish_status.latitude',
    'dish_status.longitude',
    'dish_status.altitude',

    'dish_usage.download_usage',
    'dish_usage.upload_usage',
    ]

for data_dir in data_dirs:
    print(f"Processing directory: {data_dir}")
    list_of_files = os.listdir(data_dir)
    list_of_files.sort()
    num_files = len(list_of_files)
    print(f'Number of files in {data_dir}: {num_files}')

    for file in list_of_files:
        # Skipping files that do not end with ".gzip"
        if not file.endswith(".gzip"):
            continue
        #temp_df = pd.read_parquet(os.path.join(data_dir, file), columns=starlink_selected_columns)
        temp_df = pd.read_parquet(os.path.join(data_dir, file)) # alternatively look at all columns
        df_list.append(temp_df)

# Concatenate all DataFrames
#df = pd.concat(df_list, ignore_index=True)
starlink_telemetry = pd.concat(df_list)

print("Final DataFrame shape:", starlink_telemetry.shape)
starlink_telemetry.info()
starlink_telemetry.head()

In [ ]:
starlink_telemetry = preprocess_mikrotik_starlink.preprocess_data_starlink_mqtt(starlink_telemetry)

In [ ]:
starlink_telemetry.head()

In [ ]:
starlink_telemetry.keys()

In [ ]:
starlink_telemetry[['dish_status.pop_ping_latency_ms']].describe(include='all')

In [ ]:
starlink_telemetry['dish_status.pop_ping_latency_ms'].plot(figsize=fig_size, style='.-', title='Starlink Latency (ms)')

In [ ]:
data_dir = 'data/2025-05-06-twamp/'

list_of_files = os.listdir(data_dir)
list_of_files.sort()
num_files = len(list_of_files)
print('Number of files: {}'.format(num_files))

selected_columns = [
    'identity',
    'timestamp', # added by twamp logger
    #'@timestamp', # added by mqtt logger

    'outbound.min',
    'outbound.max',
    'outbound.avg',
    'outbound.jitter',
    'outbound.loss',

    'inbound.min',
    'inbound.max',
    'inbound.avg',
    'inbound.jitter',
    'inbound.loss',

    'roundtrip.min',
    'roundtrip.max',
    'roundtrip.avg',
    'roundtrip.jitter',
    'roundtrip.loss',
]

df_list = []
for file in list_of_files:
    # Skipping the files we're not using
    if file[-5:] != ".gzip": # .gzip
        continue
    temp_df = pd.read_parquet(data_dir+file, columns=selected_columns)
    df_list.append(temp_df)
twamp_data = pd.concat(df_list)
twamp_data.info()

In [ ]:
twamp_data.head()

In [ ]:
twamp_data.tail()

In [ ]:
twamp_data['identity'].unique()

In [ ]:
twamp_data['identity'].value_counts()

In [ ]:
# Infer objects, then convert dtypes
twamp_data = twamp_data.infer_objects().convert_dtypes()

for column in twamp_data.columns:
        try:
            twamp_data[column] = pd.to_numeric(twamp_data[column])
        except (ValueError, TypeError):
            pass  # Skip columns that cannot be converted

# Rename timestamp from Python and keep it for future use
# It is unreliable if a lot of messages come at the same time due to congestion
twamp_data['timestamp_python'] = pd.to_datetime(twamp_data.pop('timestamp'))
twamp_data["timestamp_python"] = (
    twamp_data["timestamp_python"]
    .dt.tz_localize(None)        # 06:34:55.448743  (drop +02:00, keep clock time)
    + pd.Timedelta(hours=2)      # 08:34:55.448743
)
twamp_data = twamp_data.sort_values(by=['timestamp_python'])
twamp_data = twamp_data.set_index('timestamp_python', drop=False)

twamp_data.info(verbose=True, show_counts=True, memory_usage='deep')

In [ ]:
twamp_data.head()

In [ ]:
twamp_data.tail()

In [ ]:
twamp_data.keys()

In [ ]:
twamp_data['identity'].unique()

In [ ]:
# #df = df.between_time('15:00', '21:00')
# start_date = '2025-01-22 09:00:00'
# end_date = '2025-01-22 18:00:00'
# mask = (df['timestamp_python'] > start_date) & (df['timestamp_python'] <= end_date)
# df = df.loc[mask] 

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

Y_LIM = (0, 500)

twamp_data.query("identity == 'ADDIX-D2'")['roundtrip.avg'].rename('D2-RTT').plot(ax=ax, color='r', ylim=Y_LIM)
twamp_data.query("identity == 'ADDIX-DTAG'")['roundtrip.avg'].rename('DTAG-RTT').plot(ax=ax, color='g', ylim=Y_LIM)
#twamp_data.query("identity == 'CAU-4329'")['roundtrip.avg'].rename('4329-RTT').plot(ax=ax, color='b', ylim=Y_LIM)
#twamp_data.query("identity == 'CAU-0C'")['roundtrip.avg'].rename('0C-RTT').plot(ax=ax, color='m', ylim=Y_LIM)
twamp_data.query("identity == 'Starlink'")['roundtrip.avg'].rename('Starlink-RTT').plot(ax=ax, color='c', ylim=Y_LIM)
ax.axhline(twamp_data.query("identity == 'ADDIX-D2'")['roundtrip.avg'].mean(), color='r', alpha=0.7, label='D2 Mean')
ax.axhline(twamp_data.query("identity == 'ADDIX-DTAG'")['roundtrip.avg'].mean(), color='g', alpha=0.7, label='DTAG Mean')
#ax.axhline(twamp_data.query("identity == 'CAU-4329'")['roundtrip.avg'].mean(), color='b', alpha=0.7, label='4329 Mean')
#ax.axhline(twamp_data.query("identity == 'CAU-0C'")['roundtrip.avg'].mean(), color='m', alpha=0.7, label='0C Mean')
ax.axhline(twamp_data.query("identity == 'Starlink'")['roundtrip.avg'].mean(), color='c', alpha=0.7, label='Starlink Mean')
ax.set_title('RTT over time')
ax.set_ylabel('RTT [ms]')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
twamp_data_D2 = twamp_data.query("identity == 'ADDIX-D2'")
twamp_data_DTAG = twamp_data.query("identity == 'ADDIX-DTAG'")
twamp_data_Starlink = twamp_data.query("identity == 'Starlink'")
twamp_data_0C = twamp_data.query("identity == 'CAU-0C'")
twamp_data_4329 = twamp_data.query("identity == 'CAU-4329'")

In [ ]:
fig, ax = plt.subplots(figsize=(40, 10))

twamp_data_D2['roundtrip.avg'].rename('D2-RTT').plot(ax=ax, ylim=(0, 1000))
twamp_data_DTAG['roundtrip.avg'].rename('DTAG-RTT').plot(ax=ax, ylim=(0, 1000))
twamp_data_Starlink['roundtrip.avg'].rename('Starlink-RTT').plot(ax=ax, ylim=(0, 1000))
twamp_data_0C['roundtrip.avg'].rename('0C-RTT').plot(ax=ax, ylim=(0, 1000))
twamp_data_4329['roundtrip.avg'].rename('4329-RTT').plot(ax=ax, ylim=(0, 1000))
ax.set_title('RTT over time')
ax.set_ylabel('RTT [ms]')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
x1 = twamp_data_Starlink['roundtrip.avg'].dropna().to_numpy()
x2 = twamp_data_D2['roundtrip.avg'].dropna().to_numpy()
x3 = twamp_data_DTAG['roundtrip.avg'].dropna().to_numpy()
x_all = np.concatenate([x1, x2, x3])

bin_width = 1.0  # ms (pick something you can justify)
bins = np.arange(x_all.min(), x_all.max() + bin_width, bin_width)

fig, ax = plt.subplots(figsize=(4,3), dpi=200)

_, _, p1 = ax.hist(x1, bins=bins, alpha=0.5, label='Starlink', color='red', density=False)
_, _, p2 = ax.hist(x2, bins=bins, alpha=0.5, label='5G SA', color='green', density=False)
_, _, p3 = ax.hist(x3, bins=bins, alpha=0.5, label='5G NSA', color='blue', density=False)

# use the same colors as the bars
c1 = p1[0].get_facecolor()  # RGBA
c2 = p2[0].get_facecolor()
c3 = p3[0].get_facecolor()

m1 = np.median(x1)
m2 = np.median(x2)
m3 = np.median(x3)
print(f"Starlink median RTT: {m1} ms")
print(f"D2 median RTT: {m2} ms")
print(f"DTAG median RTT: {m3} ms")

ax.axvline(m1, color=c1, alpha=1.0, linestyle='--', linewidth=2, label=f'median')
ax.axvline(m2, color=c2, alpha=1.0, linestyle='--',  linewidth=2, label=f'median')
ax.axvline(m3, color=c3, alpha=1.0, linestyle='--',  linewidth=2, label=f'median')

p1 = np.percentile(x1, 90)
p2 = np.percentile(x2, 90)
p3 = np.percentile(x3, 90)
print(f"Starlink 90th percentile RTT: {p1} ms")
print(f"D2 90th percentile RTT: {p2} ms")
print(f"DTAG 90th percentile RTT: {p3} ms")

p1 = np.percentile(x1, 95)
p2 = np.percentile(x2, 95)
p3 = np.percentile(x3, 95)
print(f"Starlink 95th percentile RTT: {p1} ms")
print(f"D2 95th percentile RTT: {p2} ms")
print(f"DTAG 95th percentile RTT: {p3} ms")

mean1 = np.mean(x1)
mean2 = np.mean(x2)
mean3 = np.mean(x3)

print(f"Starlink mean RTT: {mean1} ms")
print(f"D2 mean RTT: {mean2} ms")
print(f"DTAG mean RTT: {mean3} ms")

# ax.axvline(p1, color=c1, alpha=1.0, linestyle=':', linewidth=2, label=f'p90')
# ax.axvline(p2, color=c2, alpha=1.0, linestyle=':', linewidth=2, label=f'p90')
# ax.axvline(p3, color=c3, alpha=1.0, linestyle=':', linewidth=2, label=f'p90')

# ax.axvline(twamp_data_Starlink['roundtrip.avg'].mean(), linestyle='--', alpha=0.7, color='red', label='Mean')
# ax.axvline(twamp_data_D2['roundtrip.avg'].mean(), linestyle=':',  alpha=0.7, color='green', label='Mean')
# ax.axvline(twamp_data_DTAG['roundtrip.avg'].mean(), linestyle='-.',  alpha=0.7, color='blue', label='Mean')

# Only zoom the view (does NOT change binning or the histogram computation)
ax.set_xlim(20, 70)

ax.set_xlabel('RTT (ms)')
ax.set_ylabel('Count')
ax.grid(True, linestyle='--', alpha=0.3)
ax.legend(loc='best')
plt.tight_layout()
plt.savefig("plots/TWAMP-RTT-comparison.pdf", bbox_inches="tight")
plt.savefig("plots/TWAMP-RTT-comparison.png", bbox_inches="tight")
plt.show()

In [ ]:
x1 = twamp_data_Starlink['roundtrip.max'].dropna().to_numpy()
x2 = twamp_data_D2['roundtrip.max'].dropna().to_numpy()
x3 = twamp_data_DTAG['roundtrip.max'].dropna().to_numpy()
x_all = np.concatenate([x1, x2, x3])

bin_width = 1.0  # ms (pick something you can justify)
bins = np.arange(x_all.min(), x_all.max() + bin_width, bin_width)

fig, ax = plt.subplots(figsize=(4,3), dpi=200)

_, _, p1 = ax.hist(x1, bins=bins, alpha=0.5, label='Starlink', color='red', density=False)
_, _, p2 = ax.hist(x2, bins=bins, alpha=0.5, label='5G SA', color='green', density=False)
_, _, p3 = ax.hist(x3, bins=bins, alpha=0.5, label='5G NSA', color='blue', density=False)

# use the same colors as the bars
c1 = p1[0].get_facecolor()  # RGBA
c2 = p2[0].get_facecolor()
c3 = p3[0].get_facecolor()

m1 = np.median(x1)
m2 = np.median(x2)
m3 = np.median(x3)
print(f"Starlink median RTT: {m1} ms")
print(f"D2 median RTT: {m2} ms")
print(f"DTAG median RTT: {m3} ms")

ax.axvline(m1, color=c1, alpha=1.0, linestyle='--', linewidth=2, label=f'median')
ax.axvline(m2, color=c2, alpha=1.0, linestyle='--',  linewidth=2, label=f'median')
ax.axvline(m3, color=c3, alpha=1.0, linestyle='--',  linewidth=2, label=f'median')

p1 = np.percentile(x1, 90)
p2 = np.percentile(x2, 90)
p3 = np.percentile(x3, 90)
print(f"Starlink 90th percentile RTT: {p1} ms")
print(f"D2 90th percentile RTT: {p2} ms")
print(f"DTAG 90th percentile RTT: {p3} ms")

p1 = np.percentile(x1, 95)
p2 = np.percentile(x2, 95)
p3 = np.percentile(x3, 95)
print(f"Starlink 95th percentile RTT: {p1} ms")
print(f"D2 95th percentile RTT: {p2} ms")
print(f"DTAG 95th percentile RTT: {p3} ms")

mean1 = np.mean(x1)
mean2 = np.mean(x2)
mean3 = np.mean(x3)

print(f"Starlink mean RTT: {mean1} ms")
print(f"D2 mean RTT: {mean2} ms")
print(f"DTAG mean RTT: {mean3} ms")

# ax.axvline(p1, color=c1, alpha=1.0, linestyle=':', linewidth=2, label=f'p90')
# ax.axvline(p2, color=c2, alpha=1.0, linestyle=':', linewidth=2, label=f'p90')
# ax.axvline(p3, color=c3, alpha=1.0, linestyle=':', linewidth=2, label=f'p90')

# ax.axvline(twamp_data_Starlink['roundtrip.avg'].mean(), linestyle='--', alpha=0.7, color='red', label='Mean')
# ax.axvline(twamp_data_D2['roundtrip.avg'].mean(), linestyle=':',  alpha=0.7, color='green', label='Mean')
# ax.axvline(twamp_data_DTAG['roundtrip.avg'].mean(), linestyle='-.',  alpha=0.7, color='blue', label='Mean')

# Only zoom the view (does NOT change binning or the histogram computation)
ax.set_xlim(20, 90)

ax.set_xlabel('max RTT (ms)')
ax.set_ylabel('Count')
ax.grid(True, linestyle='--', alpha=0.3)
ax.legend(loc='best')
plt.tight_layout()
plt.savefig("plots/TWAMP-max-RTT-comparison.pdf", bbox_inches="tight")
plt.savefig("plots/TWAMP-max-RTT-comparison.png", bbox_inches="tight")
plt.show()

In [ ]:
x1 = twamp_data_Starlink['roundtrip.jitter'].dropna().to_numpy()
x2 = twamp_data_D2['roundtrip.jitter'].dropna().to_numpy()
x3 = twamp_data_DTAG['roundtrip.jitter'].dropna().to_numpy()
x_all = np.concatenate([x1, x2, x3])

bin_width = 1.0  # ms (pick something you can justify)
bins = np.arange(x_all.min(), x_all.max() + bin_width, bin_width)

fig, ax = plt.subplots(figsize=(4,3), dpi=200)

_, _, p1 = ax.hist(x1, bins=bins, alpha=0.5, label='Starlink', color='red', density=False)
_, _, p2 = ax.hist(x2, bins=bins, alpha=0.5, label='5G SA', color='green', density=False)
_, _, p3 = ax.hist(x3, bins=bins, alpha=0.5, label='5G NSA', color='blue', density=False)

# use the same colors as the bars
c1 = p1[0].get_facecolor()  # RGBA
c2 = p2[0].get_facecolor()
c3 = p3[0].get_facecolor()

m1 = np.median(x1)
m2 = np.median(x2)
m3 = np.median(x3)
print(f"Starlink median Jitter: {m1} ms")
print(f"D2 median Jitter: {m2} ms")
print(f"DTAG median Jitter: {m3} ms")

ax.axvline(m1, color=c1, alpha=1.0, linestyle='--', linewidth=2, label=f'median')
ax.axvline(m2, color=c2, alpha=1.0, linestyle='--',  linewidth=2, label=f'median')
ax.axvline(m3, color=c3, alpha=1.0, linestyle='--',  linewidth=2, label=f'median')

p1 = np.percentile(x1, 90)
p2 = np.percentile(x2, 90)
p3 = np.percentile(x3, 90)

print(f"Starlink 90th percentile Jitter: {p1} ms")
print(f"D2 90th percentile Jitter: {p2} ms")
print(f"DTAG 90th percentile Jitter: {p3} ms")

p1 = np.percentile(x1, 95)
p2 = np.percentile(x2, 95)
p3 = np.percentile(x3, 95)

print(f"Starlink 95th percentile Jitter: {p1} ms")
print(f"D2 95th percentile Jitter: {p2} ms")
print(f"DTAG 95th percentile Jitter: {p3} ms")

mean1 = np.mean(x1)
mean2 = np.mean(x2)
mean3 = np.mean(x3)

print(f"Starlink mean Jitter: {mean1} ms")
print(f"D2 mean Jitter: {mean2} ms")
print(f"DTAG mean Jitter: {mean3} ms")

# ax.axvline(p1, color=c1, alpha=1.0, linestyle=':', linewidth=2, label=f'p90')
# ax.axvline(p2, color=c2, alpha=1.0, linestyle=':', linewidth=2, label=f'p90')
# ax.axvline(p3, color=c3, alpha=1.0, linestyle=':', linewidth=2, label=f'p90')

# ax.axvline(twamp_data_Starlink['roundtrip.jitter'].mean(), linestyle='--', alpha=0.7, color='red', label='Mean')
# ax.axvline(twamp_data_D2['roundtrip.jitter'].mean(), linestyle=':',  alpha=0.7, color='green', label='Mean')
# ax.axvline(twamp_data_DTAG['roundtrip.jitter'].mean(), linestyle='-.',  alpha=0.7, color='blue', label='Mean')

# Only zoom the view (does NOT change binning or the histogram computation)
ax.set_xlim(0, 15)

ax.set_xlabel('Jitter (ms)')
ax.set_ylabel('Count')
ax.grid(True, linestyle='--', alpha=0.3)
ax.legend(loc='best')
plt.tight_layout()
plt.savefig("plots/TWAMP-RTT-Jitter-comparison.pdf", bbox_inches="tight")
plt.savefig("plots/TWAMP-RTT-Jitter-comparison.png", bbox_inches="tight")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(4*1.2, 3*1.2), dpi=200)

# Prepare data (drop NaNs)
starlink_rtt = twamp_data_Starlink['roundtrip.avg'].dropna().values
vodafone_rtt = twamp_data_D2['roundtrip.avg'].dropna().values
telekom_rtt = twamp_data_DTAG['roundtrip.avg'].dropna().values

starlink_inbound = twamp_data_Starlink['inbound.avg'].dropna().values
vodafone_inbound = twamp_data_D2['inbound.avg'].dropna().values
telekom_inbound = twamp_data_DTAG['inbound.avg'].dropna().values

starlink_outbound = twamp_data_Starlink['outbound.avg'].dropna().values
vodafone_outbound = twamp_data_D2['outbound.avg'].dropna().values
telekom_outbound = twamp_data_DTAG['outbound.avg'].dropna().values

# Sort values
starlink_rtt_sorted = np.sort(starlink_rtt)
vodafone_rtt_sorted = np.sort(vodafone_rtt)
telekom_rtt_sorted = np.sort(telekom_rtt)

starlink_inbound_sorted = np.sort(starlink_inbound)
vodafone_inbound_sorted = np.sort(vodafone_inbound)
telekom_inbound_sorted = np.sort(telekom_inbound)

starlink_outbound_sorted = np.sort(starlink_outbound)
vodafone_outbound_sorted = np.sort(vodafone_outbound)
telekom_outbound_sorted = np.sort(telekom_outbound)

# ECDF values
starlink_rtt_ecdf = np.arange(1, len(starlink_rtt_sorted) + 1) / len(starlink_rtt_sorted)
vodafone_rtt_ecdf = np.arange(1, len(vodafone_rtt_sorted) + 1) / len(vodafone_rtt_sorted)
telekom_rtt_ecdf = np.arange(1, len(telekom_rtt_sorted) + 1) / len(telekom_rtt_sorted)

starlink_inbound_ecdf = np.arange(1, len(starlink_inbound_sorted) + 1) / len(starlink_inbound_sorted)
vodafone_inbound_ecdf = np.arange(1, len(vodafone_inbound_sorted) + 1) / len(vodafone_inbound_sorted)
telekom_inbound_ecdf = np.arange(1, len(telekom_inbound_sorted) + 1) / len(telekom_inbound_sorted)

starlink_outbound_ecdf = np.arange(1, len(starlink_outbound_sorted) + 1) / len(starlink_outbound_sorted)
vodafone_outbound_ecdf = np.arange(1, len(vodafone_outbound_sorted) + 1) / len(vodafone_outbound_sorted)
telekom_outbound_ecdf = np.arange(1, len(telekom_outbound_sorted) + 1) / len(telekom_outbound_sorted)

# Plot ECDFs (capture lines to reuse colors)
(l1_rtt,) = ax.step(starlink_rtt_sorted, starlink_rtt_ecdf, where='post', alpha=0.5, label='Starlink', color='red')
(l2_rtt,) = ax.step(vodafone_rtt_sorted, vodafone_rtt_ecdf, where='post', alpha=0.5, label='5G SA', color='green')
(l3_rtt,) = ax.step(telekom_rtt_sorted, telekom_rtt_ecdf, where='post', alpha=0.5, label='5G NSA', color='blue')

(l1_in,) = ax.step(starlink_inbound_sorted, starlink_inbound_ecdf, where='post', alpha=0.2, label='Starlink Inbound', color='red', linestyle='--')
(l2_in,) = ax.step(vodafone_inbound_sorted, vodafone_inbound_ecdf, where='post', alpha=0.2, label='5G SA Inbound', color='green', linestyle='--')
(l3_in,) = ax.step(telekom_inbound_sorted, telekom_inbound_ecdf, where='post', alpha=0.2, label='5G NSA Inbound', color='blue', linestyle='--')

(l1_out,) = ax.step(starlink_outbound_sorted, starlink_outbound_ecdf, where='post', alpha=0.2, label='Starlink Outbound', color='red', linestyle=':')
(l2_out,) = ax.step(vodafone_outbound_sorted, vodafone_outbound_ecdf, where='post', alpha=0.2, label='5G SA Outbound', color='green', linestyle=':')
(l3_out,) = ax.step(telekom_outbound_sorted, telekom_outbound_ecdf, where='post', alpha=0.2, label='5G NSA Outbound', color='blue', linestyle=':')  

c1_rtt, c2_rtt, c3_rtt = l1_rtt.get_color(), l2_rtt.get_color(), l3_rtt.get_color()
c1_in, c2_in, c3_in = l1_in.get_color(), l2_in.get_color(), l3_in.get_color()
c1_out, c2_out, c3_out = l1_out.get_color(), l2_out.get_color(), l3_out.get_color()

# --- stats (median + p95) ---
s_rtt_med, s_rtt_p90 = np.percentile(starlink_rtt, [50, 90])
v_rtt_med, v_rtt_p90 = np.percentile(vodafone_rtt, [50, 90])
t_rtt_med, t_rtt_p90 = np.percentile(telekom_rtt, [50, 90])

# ax.axvline(s_rtt_med, color=c1_rtt, linestyle='--', linewidth=1.6, alpha=0.9,
#            label=f'median')
# ax.axvline(v_rtt_med, color=c2_rtt, linestyle='--', linewidth=1.6, alpha=0.9,
#            label=f'median')
# ax.axvline(t_rtt_med, color=c3_rtt, linestyle='--', linewidth=1.6, alpha=0.9,
#            label=f'median')
# ----------------------------

x_all = np.concatenate([starlink_rtt, vodafone_rtt, telekom_rtt])

lo, hi = np.percentile(x_all, [0.5, 95])   # try [0, 99] or [1, 99] too
pad = 0.02 * (hi - lo)

#ax.set_xlim(max(0, lo - pad), hi + pad)      # clamp at 0 for RTT/jitter
ax.set_xlim(0, 200)
ax.set_xlabel('RTT [ms]')
ax.set_ylabel('Cumulative probability')
ax.grid(True, linestyle='--', alpha=0.3)
ax.legend(loc='best')
plt.tight_layout()
#plt.savefig("plots/TWAMP-RTT-ecdf-comparison.pdf", bbox_inches="tight")
#plt.savefig("plots/TWAMP-RTT-ecdf-comparison.png", bbox_inches="tight")
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=200)

# Prepare data (drop NaNs)
starlink = twamp_data_Starlink['roundtrip.avg'].dropna().values
vodafone = twamp_data_D2['roundtrip.avg'].dropna().values
telekom = twamp_data_DTAG['roundtrip.avg'].dropna().values

# Sort values
starlink_sorted = np.sort(starlink)
vodafone_sorted = np.sort(vodafone)
telekom_sorted = np.sort(telekom)

# ECDF values
starlink_ecdf = np.arange(1, len(starlink_sorted) + 1) / len(starlink_sorted)
vodafone_ecdf = np.arange(1, len(vodafone_sorted) + 1) / len(vodafone_sorted)
telekom_ecdf = np.arange(1, len(telekom_sorted) + 1) / len(telekom_sorted)

# Plot ECDFs (capture lines to reuse colors)
(l1,) = ax.step(starlink_sorted, starlink_ecdf, where='post', alpha=0.5, label='Starlink', color='red')
(l2,) = ax.step(vodafone_sorted, vodafone_ecdf, where='post', alpha=0.5, label='5G SA', color='green')
(l3,) = ax.step(telekom_sorted, telekom_ecdf, where='post', alpha=0.5, label='5G NSA', color='blue')

c1, c2, c3 = l1.get_color(), l2.get_color(), l3.get_color()

# --- stats (median + p95) ---
s_med, s_p95 = np.percentile(starlink, [50, 90])
v_med, v_p95 = np.percentile(vodafone, [50, 90])
t_med, t_p95 = np.percentile(telekom, [50, 90])

ax.axvline(s_med, color=c1, linestyle='--', linewidth=1.6, alpha=0.9,
           label=f'median')
ax.axvline(v_med, color=c2, linestyle='--', linewidth=1.6, alpha=0.9,
           label=f'median')
ax.axvline(t_med, color=c3, linestyle='--', linewidth=1.6, alpha=0.9,
           label=f'median')

ax.axvline(s_p95, color=c1, linestyle=':', linewidth=1.6, alpha=0.9,
           label=f'p90')
ax.axvline(v_p95, color=c2, linestyle=':', linewidth=1.6, alpha=0.9,
           label=f'p90')
ax.axvline(t_p95, color=c3, linestyle=':', linewidth=1.6, alpha=0.9,
           label=f'p90')
# ----------------------------

x_all = np.concatenate([starlink, vodafone, telekom])

lo, hi = np.percentile(x_all, [0.5, 95])   # try [0, 99] or [1, 99] too
pad = 0.02 * (hi - lo)

#ax.set_xlim(max(0, lo - pad), hi + pad)      # clamp at 0 for RTT/jitter
ax.set_xlim(0, 200)
ax.set_xlabel('RTT [ms]')
ax.set_ylabel('Cumulative probability')
ax.grid(True, linestyle='--', alpha=0.3)
ax.legend(loc='best')
plt.tight_layout()
#plt.savefig("plots/TWAMP-RTT-ecdf-comparison.pdf", bbox_inches="tight")
#plt.savefig("plots/TWAMP-RTT-ecdf-comparison.png", bbox_inches="tight")
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=200)

# Prepare data (drop NaNs)
starlink = twamp_data_Starlink['inbound.avg'].dropna().values
vodafone = twamp_data_D2['inbound.avg'].dropna().values
telekom = twamp_data_DTAG['inbound.avg'].dropna().values

# Sort values
starlink_sorted = np.sort(starlink)
vodafone_sorted = np.sort(vodafone)
telekom_sorted = np.sort(telekom)

# ECDF values
starlink_ecdf = np.arange(1, len(starlink_sorted) + 1) / len(starlink_sorted)
vodafone_ecdf = np.arange(1, len(vodafone_sorted) + 1) / len(vodafone_sorted)
telekom_ecdf = np.arange(1, len(telekom_sorted) + 1) / len(telekom_sorted)

# Plot ECDFs (capture lines to reuse colors)
(l1,) = ax.step(starlink_sorted, starlink_ecdf, where='post', alpha=0.5, label='Starlink', color='red')
(l2,) = ax.step(vodafone_sorted, vodafone_ecdf, where='post', alpha=0.5, label='5G SA', color='green')
(l3,) = ax.step(telekom_sorted, telekom_ecdf, where='post', alpha=0.5, label='5G NSA', color='blue')

c1, c2, c3 = l1.get_color(), l2.get_color(), l3.get_color()

# --- stats (median + p95) ---
s_med, s_p95 = np.percentile(starlink, [50, 90])
v_med, v_p95 = np.percentile(vodafone, [50, 90])
t_med, t_p95 = np.percentile(telekom, [50, 90])

ax.axvline(s_med, color=c1, linestyle='--', linewidth=1.6, alpha=0.9,
           label=f'median')
ax.axvline(v_med, color=c2, linestyle='--', linewidth=1.6, alpha=0.9,
           label=f'median')
ax.axvline(t_med, color=c3, linestyle='--', linewidth=1.6, alpha=0.9,
           label=f'median')

ax.axvline(s_p95, color=c1, linestyle=':', linewidth=1.6, alpha=0.9,
           label=f'p90')
ax.axvline(v_p95, color=c2, linestyle=':', linewidth=1.6, alpha=0.9,
           label=f'p90')
ax.axvline(t_p95, color=c3, linestyle=':', linewidth=1.6, alpha=0.9,
           label=f'p90')
# ----------------------------

x_all = np.concatenate([starlink, vodafone, telekom])

lo, hi = np.percentile(x_all, [0.5, 95])   # try [0, 99] or [1, 99] too
pad = 0.02 * (hi - lo)

ax.set_xlim(max(0, lo - pad), hi + pad)      # clamp at 0 for RTT/jitter
#ax.set_xlim(25, 200)
ax.set_xlabel('inbound [ms]')
ax.set_ylabel('Cumulative probability')
ax.grid(True, linestyle='--', alpha=0.3)
ax.legend(loc='best')
plt.tight_layout()
#plt.savefig("plots/TWAMP-inbound-ecdf-comparison.pdf", bbox_inches="tight")
#plt.savefig("plots/TWAMP-inbound-ecdf-comparison.png", bbox_inches="tight")
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=200)

# Prepare data (drop NaNs)
starlink = twamp_data_Starlink['outbound.avg'].dropna().values
vodafone = twamp_data_D2['outbound.avg'].dropna().values
telekom = twamp_data_DTAG['outbound.avg'].dropna().values

# Sort values
starlink_sorted = np.sort(starlink)
vodafone_sorted = np.sort(vodafone)
telekom_sorted = np.sort(telekom)

# ECDF values
starlink_ecdf = np.arange(1, len(starlink_sorted) + 1) / len(starlink_sorted)
vodafone_ecdf = np.arange(1, len(vodafone_sorted) + 1) / len(vodafone_sorted)
telekom_ecdf = np.arange(1, len(telekom_sorted) + 1) / len(telekom_sorted)

# Plot ECDFs (capture lines to reuse colors)
(l1,) = ax.step(starlink_sorted, starlink_ecdf, where='post', alpha=0.5, label='Starlink', color='red')
(l2,) = ax.step(vodafone_sorted, vodafone_ecdf, where='post', alpha=0.5, label='5G SA', color='green')
(l3,) = ax.step(telekom_sorted, telekom_ecdf, where='post', alpha=0.5, label='5G NSA', color='blue')

c1, c2, c3 = l1.get_color(), l2.get_color(), l3.get_color()

# --- stats (median + p95) ---
s_med, s_p95 = np.percentile(starlink, [50, 90])
v_med, v_p95 = np.percentile(vodafone, [50, 90])
t_med, t_p95 = np.percentile(telekom, [50, 90])

ax.axvline(s_med, color=c1, linestyle='--', linewidth=1.6, alpha=0.9,
           label=f'median')
ax.axvline(v_med, color=c2, linestyle='--', linewidth=1.6, alpha=0.9,
           label=f'median')
ax.axvline(t_med, color=c3, linestyle='--', linewidth=1.6, alpha=0.9,
           label=f'median')

ax.axvline(s_p95, color=c1, linestyle=':', linewidth=1.6, alpha=0.9,
           label=f'p90')
ax.axvline(v_p95, color=c2, linestyle=':', linewidth=1.6, alpha=0.9,
           label=f'p90')
ax.axvline(t_p95, color=c3, linestyle=':', linewidth=1.6, alpha=0.9,
           label=f'p90')
# ----------------------------

x_all = np.concatenate([starlink, vodafone, telekom])

lo, hi = np.percentile(x_all, [0.5, 95])   # try [0, 99] or [1, 99] too
pad = 0.02 * (hi - lo)

ax.set_xlim(max(0, lo - pad), hi + pad)      # clamp at 0 for RTT/jitter
#ax.set_xlim(25, 200)
ax.set_xlabel('outbound [ms]')
ax.set_ylabel('Cumulative probability')
ax.grid(True, linestyle='--', alpha=0.3)
ax.legend(loc='best')
plt.tight_layout()
#plt.savefig("plots/TWAMP-outbound-ecdf-comparison.pdf", bbox_inches="tight")
#plt.savefig("plots/TWAMP-outbound-ecdf-comparison.png", bbox_inches="tight")
plt.show()


In [ ]:
twamp_data_Starlink = twamp_data_Starlink.between_time('09:00', '14:40')
starlink_telemetry = starlink_telemetry.between_time('09:00', '14:40')

df_D2 = df_D2.between_time('09:00', '14:40')
df_DTAG = df_DTAG.between_time('09:00', '14:40')

In [ ]:
twamp_data_Starlink.head()

In [ ]:
twamp_data_Starlink.tail()

In [ ]:
starlink_telemetry.head()

In [ ]:
starlink_telemetry.tail()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5), dpi=200)

twamp_data_Starlink['roundtrip.avg'].rename('TWAMP-RTT').plot(ax=ax, style='.-')
starlink_telemetry['dish_status.pop_ping_latency_ms'].rename('PoP-Latency').plot(ax=ax, style='.-')   

# twamp_data_D2['roundtrip.avg'].rename('D2-RTT').plot(ax=ax, ylim=(0, 1000))
# twamp_data_DTAG['roundtrip.avg'].rename('DTAG-RTT').plot(ax=ax, ylim=(0, 1000))
# twamp_data_Starlink['roundtrip.avg'].rename('Starlink-RTT').plot(ax=ax, ylim=(0, 1000))
# twamp_data_0C['roundtrip.avg'].rename('0C-RTT').plot(ax=ax, ylim=(0, 1000))
# twamp_data_4329['roundtrip.avg'].rename('4329-RTT').plot(ax=ax, ylim=(0, 1000))

#ax.set_title('RTT over time')
ax.set_ylabel('RTT [ms]')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend(loc='best')
plt.ylim(0, 500)
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

df_D2['speed'].rename('D2-GPS-speed').plot(ax=ax, )
df_DTAG['speed'].rename('DTAG-GPS-speed').plot(ax=ax, )

ax.set_title('GPS Speed over time')
ax.set_ylabel('GPS Speed (km/h)')
ax.grid(True, which='both', linestyle='--', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# df_cqi: columns ["ts", "cqi"]   (cqi in 1..15)
# df_lat: columns ["ts", "latency_ms"]

#lSinr

df_cqi = df_D2[['lCqi', 'timestamp_router']].copy()
df_lat = twamp_data_D2[['roundtrip.min', 'timestamp_python']].copy()

In [ ]:
df_cqi["ts"] = pd.to_datetime(df_cqi["timestamp_router"], utc=True)
df_lat["ts"] = pd.to_datetime(df_lat["timestamp_python"], utc=True)

In [ ]:
df_cqi = df_cqi.sort_values("ts")
df_lat = df_lat.sort_values("ts")

In [ ]:
# Match each latency sample to the nearest CQI sample (within a tolerance)
df_cqi_vs_latency = pd.merge_asof(
    df_lat, df_cqi,
    on="ts",
    direction="nearest",
    tolerance=pd.Timedelta("250ms")  # adjust to your sampling rates
)

df_cqi_vs_latency = df_cqi_vs_latency.dropna(subset=["lCqi", "roundtrip.min"])
df_cqi_vs_latency["lCqi"] = df_cqi_vs_latency["lCqi"].astype(int)
df_cqi_vs_latency = df_cqi_vs_latency[df_cqi_vs_latency["lCqi"].between(1, 15)]

In [ ]:
def q(x, p): 
    return x.quantile(p)

stats = (
    df_cqi_vs_latency.groupby("lCqi")["roundtrip.min"]
      .agg(
          n="size",
          mean="mean",
          median="median",
          p10=lambda s: q(s, 0.10),
          p90=lambda s: q(s, 0.90),
      )
      .reindex(range(1, 16))
)

stats

In [ ]:
x = stats.index.values
mean = stats["mean"].values
p10 = stats["p10"].values
p90 = stats["p90"].values

fig, ax = plt.subplots(figsize=(4, 3), dpi=200)

ax.plot(x, mean, marker="o", label="Mean RTT")
#ax.fill_between(x, p10, p90, alpha=0.2, label="10-90th percentile")

#ax.set_yscale("log")
#ax.set_ylim(10, 10000)
ax.set_xticks(range(1, 16))
ax.set_xlabel("CQI")
ax.set_ylabel("RTT (ms)")
ax.grid(True, which="both", linestyle="--", alpha=0.3)
ax.legend()
plt.tight_layout()
# plt.savefig("plots/WebRTC-CQI-vs-mean-RTT.pdf", bbox_inches="tight")
# plt.savefig("plots/WebRTC-CQI-vs-mean-RTT.png", bbox_inches="tight")
plt.show()

In [ ]:
starlink_telemetry_low_snr = starlink_telemetry.query("`dish_status.is_snr_above_noise_floor` == 0")
starlink_telemetry_low_snr.info()

In [ ]:
starlink_telemetry_high_snr = starlink_telemetry.query("`dish_status.is_snr_above_noise_floor` == 1")
starlink_telemetry_high_snr.info()

In [ ]:
starlink_telemetry_high_snr['dish_status.pop_ping_latency_ms'].describe()

In [ ]:
starlink_telemetry_low_snr['dish_status.pop_ping_latency_ms'].describe()

In [ ]:
starlink_telemetry_low_snr['dish_status.pop_ping_latency_ms'].head()

In [ ]:
ts = starlink_telemetry_low_snr.index

In [ ]:
twamp_data_Starlink_sel = twamp_data_Starlink.loc[twamp_data_Starlink.index.isin(ts)]

In [ ]:
twamp_data_Starlink_sel.info()

In [ ]:
twamp_data_Starlink.head()

In [ ]:
# sort required for merge_asof

# ensure both are datetime
starlink_telemetry_low_snr["ts"] = pd.to_datetime(starlink_telemetry_low_snr["timestamp"])
twamp_data_Starlink["ts"] = pd.to_datetime(twamp_data_Starlink["timestamp_python"])

a = starlink_telemetry_low_snr
b = twamp_data_Starlink

twamp_data_Starlink_matched = pd.merge_asof(
    a[["ts"]], b, on="ts",
    direction="nearest",
    tolerance=pd.Timedelta("1000ms")   # set what makes sense for you
)

In [ ]:
twamp_data_Starlink_matched.info()

In [ ]:
twamp_data_Starlink_matched['roundtrip.avg'].describe()

In [ ]:
twamp_data_Starlink['roundtrip.avg'].describe()